In [1]:
from pandas import read_excel
read_excel('./Welding Data Set_01.xlsx')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
## 필요한 패키지 설치
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch
import numpy as np
import pandas as pd
import sklearn.metrics as metric
from sklearn import preprocessing
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
##pandas의 read_excel함수를 사용하여 데이터 불러옴, 또한 index_col = “열 이름” 을 사용하여 인덱스 열 설정
welding_data = pd.read_excel("./Welding Data Set_01.xlsx", index_col ="idx")
welding_data.head()
#결과는 아래에서 확인 가능하다.

In [ ]:
## 데이터의 특성들이 어떤 값들을 가지고 있으며 몇 개씩 가지고 있는지 확인
for feature in welding_data:
     print(feature, welding_data[feature].value_counts()) #결과는 아래에서 확인 가능하다.

In [ ]:
welding_data.describe()  #결과는 아래에서 확인 가능하다.

In [ ]:
welding_data.corr()  #결과는 아래에서 확인 가능하다.

In [ ]:
## 히스토그램을 그리기 위한 for 구문
b = [15,15,15,15,15,25,13,17,15]
for index, value in enumerate(welding_data.columns):
    plt.figure(index)
    plt.hist(welding_data[value], bins = b[index], facecolor = (144 /255,171 /255,221 /255))
    plt.title(value)


In [ ]:
## 용접기 데이터에서 필요없는 부분(생산품목, 작업시간, 소재두께)들을 제외
new_welding_data = welding_data.iloc[:, 5:]
new_welding_data.head()  #결과는 아래에서 확인 가능하다.

In [ ]:
## sklearn의 preprocessing모듈에 들어있는 MinMaxScaler함수를 이용해 정규화 적용
scaler = preprocessing.MinMaxScaler()
scaler.fit(new_welding_data)
scaled_data = scaler.transform(new_welding_data)

In [ ]:
## AutoEncoder 클래스 구현 
class AutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AutoEncoder, self).__init__()
        ## initialize
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        ##오토인코더 구현
        self.AutoEncoder = nn.Sequential(
        ## 인코더 부분
        nn.Linear(input_size, hidden_size[0]),
        nn.RReLU(),
        nn.Linear(hidden_size[0], output_size),
        nn.RReLU(),
        ## 디코더 부분
        nn.Linear(output_size, hidden_size[0]),
        nn.RReLU(),
        nn.Linear(hidden_size[0], input_size)
        )
    def forward(self, inputs): 
        output =self.AutoEncoder(inputs)
   
        return output

In [ ]:
##기존이 데이터를 텐서 형태로 변환, 그리고 훈련세트와 테스트세트로 나눔
train_data = torch.Tensor(scaled_data[:8470]) ## [:8470] 처음부터 8469번까지의 데이터를 훈련세트로 지정  
print(len(train_data))
test_data = torch.Tensor(scaled_data[8470:]) ## [8470:] 8470번째 데이터부터 끝까지를 테스트세트로 지정
print(len(test_data))  #결과는 아래에서 확인 가능하다.

In [ ]:
## 훈련 하이퍼파라미터
epoch =50
batch_size =64
lr =0.01
## 모델 하이퍼파라미터
input_size =len(train_data[0])
hidden_size = [3]
output_size =2
## 손실 함수로 제곱근 오차 사용
criterion = nn.MSELoss()
## 매개변수 조정 방식으로 Adam사용
optimizer = torch.optim.Adam
##오토인코더 정의
AutoEncoder = AutoEncoder(input_size, hidden_size, output_size) 

In [ ]:
## 학습 함수에 대한 정의
def train_net(AutoEncoder, data, criterion, epochs, lr_rate =0.01):
## Optimizer에 대한 정의
    optim = optimizer(AutoEncoder.parameters(), lr =lr_rate)
## 배치 학습을 시키기 위한 데이터 변환
    data_iter = DataLoader(data, batch_size =batch_size, shuffle =True)
## 에포크 학습
    for epoch in range(1, epochs +1):
        running_loss =0.0
        for x in data_iter:
            ## 매개변수 0으로 초기화
            optim.zero_grad()
            output = AutoEncoder(x)
            ## 입력값과 출력값간의 차이인 손실값 
            loss = criterion(x, output)
            ## 손실값을 기준으로 매개변수 조정
            loss.backward()
            optim.step()
            running_loss += loss.item()
            
        ## 각 에포크마다 손실 값 표기
        print("epoch: {}, loss: {:.2f}".format(epoch, running_loss))
    return AutoEncoder

In [ ]:
## 학습 함수를 이용한 오토인코더 학습
AutoEncoder = train_net(AutoEncoder, train_data, criterion, epoch, lr)

In [ ]:
## 훈련세트의 손실값 이용한 임계값 정의
train_loss_chart = []
for data in train_data:
    output = AutoEncoder(data)
    loss = criterion(output, data)
    train_loss_chart.append(loss.item())
    
threshold = np.mean(train_loss_chart) + np.std(train_loss_chart)*4
print("Threshold :", threshold)  #결과는 아래에서 확인 가능하다.

In [ ]:
## 훈련세트의 손실값 이용한 임계값 정의
test_loss_chart = []
for data in train_data:
    output = AutoEncoder(data)
    loss = criterion(output, data)
    test_loss_chart.append(loss.item())

outlier = list(test_loss_chart >= threshold)
outlier.count(True)  #결과는 아래에서 확인 가능하다.